# Algorithmic Data Science Report

In [2]:
import numpy as np
import time

#Method to time the runtime of methods, provided by Dr Adam Barrett in the module lab
def timeit(somefunc,*args,repeats=100,**kwargs):
    times=[]
    for i in range(repeats):
        starttime=time.time()
        ans=somefunc(*args,**kwargs)
        endtime=time.time()
        timetaken=endtime-starttime
        times.append(timetaken)
    
    mean=np.mean(times)
    stdev=np.std(times)
    error=stdev/(repeats**0.5)
 
    return (ans,mean,error)

In [3]:
import re
import pandas as pd
import nltk
from nltk.tokenize import WordPunctTokenizer
nltk.download('stopwords')
from nltk.corpus import stopwords
# needed for nltk.pos_tag function nltk.download(’averaged_perceptron_tagger’)
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [4]:
import plotly.express as px
import plotly.graph_objects as go

In [5]:
#My method for getting the word tokens from a text document
def get_tokens(text):
    #tokenize
    word_punct_token = WordPunctTokenizer().tokenize(text)
    
    #normalize tokens
    clean_token=[]
    for token in word_punct_token:
        token = token.lower()
        # remove any value that are not alphabetical
        new_token = re.sub(r'[^a-zA-Z]+', '', token) 
        # remove empty value and single character value
        if new_token != "" and len(new_token) >= 2: 
            vowels=len([v for v in new_token if v in "aeiou"])
            if vowels != 0: # remove line that only contains consonants
                clean_token.append(new_token)
    
    # Get the list of stop words
    stop_words = stopwords.words('english')
    # Remove the stopwords from the list of tokens
    tokens = [x for x in clean_token if x not in stop_words]
    
    return tokens

In [6]:
#My test document, the complete first volume of Lord of the Rings: The Fellowship of the Ring
f = open('Lotr_Fellowship.txt', encoding="utf8")
text=f.read()
f.close()

#Get tokens
tokens=get_tokens(text)

In [7]:
import random
#Here I set the random seed for my use of the random library throughout the rest of this notebook
random.seed(0)

#Method to turn a "bag" of tokens from a sample into a dictionary
def make_dict(tokenlist):
    res_dict={}
    for token in tokenlist:
        res_dict[token]=res_dict.get(token,0)+1
    return res_dict

#Method to create x "bag" samples with y words per "bag"
def create_doc_dict_samples(no_of_bags,words_per_bag):
    bags=[]
    for doc in range(no_of_bags):
        bags.append(random.sample(tokens, words_per_bag))
    
    doc_dicts=[]
    for bag in bags:
        doc_dicts.append(make_dict(bag))
        
    return doc_dicts

In [8]:
#Here I create all my sample documents from my one larger document
doc_dicts=[]
for i in range(100,10000,100):
    doc_dicts=doc_dicts+create_doc_dict_samples(10,i)

#Theoretical and empirical analysis of the runtime of Jaccard’s similarity measure applied to large documents represented as bags of words (in a Python dictionary)



The theoretical runtime for Jaccard's measure, as I have written it below, is O(n), with n being the size of the larger dictionary input. It would be the size of the dictionary used in the for loop for the method given, however we need to use the method "maketotal" to sum the values in each dictionary, which is order k where k is the size of the dictionary being summed. Also, dictionary.get() is O(1) in python (Tas, 2021), so the for loop in the main algorithm is just the order of the dictionary looped through, as is the case with the cost of looking up a value with dictionary[item] in the "maketotal" method.

The following is my code for the Jaccard algorithm and the "maketotal" method used within.

In [72]:
#Sum dictionary values
def maketotal(dict1):
    total=0
    for item in dict1:
        total += dict1[item]
    return total

#Jaccard method from module labs by Dr Adam Barrett
def jaccard(C1, C2):
    intersection=0
    #Loop through C1
    for token,c1_val in C1.items():
        #Use .get() which is O(1)
        c2_val=C2.get(token,0)
        if c2_val!=0:
            intersection+=min(c1_val,c2_val)
    #Use sum() which is O(1)
    union=maketotal(C1)+maketotal(C2)-intersection
    return intersection/union

In [73]:
#Here I time my Jaccard method and create a dataframe with the results
jaccard_c1_lengths=[]
jaccard_c1_times=[]
jaccard_c1_errors=[]

for i in range(500):
    #Randomly sample from my documents
    c1=random.choice(doc_dicts)
    c2=random.choice(doc_dicts)
    jaccard_c1_lengths.append(max(len(c1),len(c2)))
    #Time
    (ans_c1,mean_c1,error_c1)=timeit(jaccard,c1,c2,repeats=100)
    jaccard_c1_times.append(mean_c1)
    jaccard_c1_errors.append(error_c1)
    
#Make dataframe
jaccard_dict={'standard_n':jaccard_c1_lengths, 'standard_jaccard_time':jaccard_c1_times, 'standard_jaccard_error':jaccard_c1_errors}    
jaccard_df=pd.DataFrame(jaccard_dict)

Here I plot the mean time taken for 100 repeats of Jaccard's algorithm on randomly selected dictionaries of various sizes. I take dictionary size n to be the largest dictionary in the input arguments.

In [74]:
#Plotting
fig0 = go.Figure()
fig0.add_trace(go.Scatter(x=jaccard_df["standard_n"], y=jaccard_df["standard_jaccard_time"],mode='markers',name='regular Jaccard'))
fig0.update_layout(title="Regular Jaccard Algorithm Time(s) vs Dictionary Size n",xaxis_title="n",yaxis_title="time(s)")
fig0.show()

To analyse the runtime of Jaccard empirically, I have plotted a log-log plot below, and output the OLS regression results.

In [75]:
#Plotting
fig2 = px.scatter(x=np.log(jaccard_df[jaccard_df["standard_jaccard_time"]>0]["standard_n"]), y=np.log(jaccard_df[jaccard_df["standard_jaccard_time"]>0]["standard_jaccard_time"]), trendline="ols")
fig2.update_layout(title="Regular Jaccard Algorithm log-log plot",xaxis_title="ln(n)", yaxis_title="ln(time(s))")
fig2.show()
results2 = px.get_trendline_results(fig2)
results2.px_fit_results.iloc[0].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.666
Model:                            OLS   Adj. R-squared:                  0.665
Method:                 Least Squares   F-statistic:                     991.1
Date:                Thu, 09 Dec 2021   Prob (F-statistic):          1.55e-120
Time:                        17:42:58   Log-Likelihood:                -73.616
No. Observations:                 500   AIC:                             151.2
Df Residuals:                     498   BIC:                             159.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        -16.2890      0.298    -54.679      0.000     -16.874     -15.704
x1             1.2141      0.039     31.482      0.000       1.138       1.290
==============================================================================
Omnibus:                      161.610   Durbin-Watson:                   2.150
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              416.996
Skew:                          -1.623   Prob(JB):                     2.82e-91
Kurtosis:                       6.080   Cond. No.                         186.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

My results showed that the regression line for the log-log plot for Jaccard had the equation $y=1.2141x-16.2890$. It therefore has gradient significantly greater than 1 and as such, doesn't support the theoretical conclusion that it has a runtime of order n or less, with n being the size of the larger dictionary compared. The constant for my computer for Jaccard here was $e^{-16.2890}=8.42902194\cdot10^{-8}$.

#Theoretical and empirical analysis of the runtime of the cosine similarity measure applied to documents represented as (dense representation) vectors

My dense vector representation cosine similarity algorithm are O(n) where n is the length of the vectors being compared as the algorithm uses only 1 for loop, similarly to my Jaccard methods.

The following is my code for both dense cosine similarity algorithms, one taking lists as arguments and using a for loop, and one taking numpy arrays and using numpy dot product. Code is also included for converting the list of dictionaries representation into a matrix, however in my "matrix" I have actually created vector representations of each bag separately at first, so that they don't take all keys and have varying lengths. This allows me to run a similarity comparison for each with itself (which has no impact on runtime for dense cosine as the actual values within are irrelevant) to empirically analyse time complexity.

In [18]:
#This method was provided in the module labs by Dr Adam Barrett to turn a dictionary representation of a "bag" of words
#into a dense vector representation, stored as a matrix
def make_matrix(list_of_dicts):
    #first of all make a list of all of the features that occur in any document - these will be the dimensions of the matrix
    allfeatures={}    
    for docdict in list_of_dicts:
        for feat in docdict.keys():
            allfeatures[feat]=1
    
    dimensions=list(allfeatures.keys())
    #don't strictly need to sort it - but it is good practise to make sure it is reproducible
    sorted(dimensions)
    
    matrix=[]
    #each row in the matrix will be one of the dimensions
    for dimension in dimensions:
        row=[]
        #look up the appropriate value for each document
        for docdict in list_of_dicts:
            row.append(docdict.get(dimension,0)) #this will append the document's value if present, 0 otherwise
        matrix.append(row)
        
    return matrix

#This method was also provided to transpose said matrix
def transpose(matrix):
    transposed=[]
    for i in range(0,len(matrix[0])):
        transposed.append([row[i] for row in matrix])
        
    return transposed

In [19]:
#This is my naive cosine method for dense vector representations of "bags" of words
def naiveCosine(a,b):
    num=0
    d1=0
    d2=0
    for i in range(len(a)):
        #a dot b
        num+=a[i]*b[i]
        #a dot a
        d1+=a[i]*a[i]
        #b dot b
        d2+=b[i]*b[i]
    return num/(d1*d2)**0.5

#Numpy version of the above
def naiveCosineNumpyDot(a_np,b_np):
    return np.dot(a_np,b_np)/np.power(np.dot(a_np,a_np)*np.dot(b_np,b_np),0.5)

In [20]:
#Here I create my jagged list of transposed vectors, as I want different sizes for comparison
m=[]
for i in range(len(doc_dicts)):
  m_i = transpose(make_matrix([doc_dicts[i]]))
  m = m + m_i

In [21]:
#Here I time both cosine measures as detailed above
ns=[]
cosine_times=[]
cosine_errors=[]
numpy_dot_cosine_times=[]
numpy_dot_cosine_errors=[]

for i in range(500):
  vector=random.choice(range(0,989))
  a=m[vector]
  b=a
  a_np=np.array(m[vector])
  b_np=a_np
  ns.append(len(a))
  (ans_cos,mean_cos,error_cos)=timeit(naiveCosine,a,b,repeats=100)
  (ans_np,mean_np,error_np)=timeit(naiveCosineNumpyDot,a_np,b_np,repeats=100)
  cosine_times.append(mean_cos)
  cosine_errors.append(error_cos)
  numpy_dot_cosine_times.append(mean_np)
  numpy_dot_cosine_errors.append(error_np)
    
cosine_dict={'n':ns, 'naive_cosine_time':cosine_times, 'numpy_naive_cosine_time':numpy_dot_cosine_times, 'naive_cosine_error':cosine_errors, 'numpy_naive_cosine_error':numpy_dot_cosine_errors}    
cosine_df=pd.DataFrame(cosine_dict)

Here I plot an overlay of the mean time taken for 100 repeats of both versions of cosine similarity algorithm on randomly selected vectors of various sizes. For both my list version, hereby referred to as "Naive Cosine", and the numpy version, the vector lengths are as stated, the lengths of the vectors used in both arguments, which have to be equal.

In [22]:
#Plotting
fig4 = go.Figure()
fig4.add_trace(go.Scatter(x=cosine_df["n"], y=cosine_df["naive_cosine_time"],mode='markers',name='naive cosine'))
fig4.add_trace(go.Scatter(x=cosine_df["n"], y=cosine_df["numpy_naive_cosine_time"],mode='markers',name='numpy cosine'))
fig4.update_layout(title="Comparison of Naive and Numpy Cosine Algorithms",xaxis_title="vector length", yaxis_title="time(s)")
fig4.show()

From the results, Numpy Cosine is clearly significantly faster. This is because NumPy completes the calculation in lower level language C, before passing the result back to Python (Yanchii, 2019). Below I have produced log-log plots for both methods and the OLS regression results for empirical runtime analysis.

In [23]:
#Plotting
fig5 = px.scatter(x=np.log(cosine_df[cosine_df["naive_cosine_time"]>0]["n"]), y=np.log(cosine_df[cosine_df["naive_cosine_time"]>0]["naive_cosine_time"]), trendline="ols")
fig5.update_layout(title="Log-log Plot for Naive Cosine",xaxis_title="log(vector length)", yaxis_title="log(time(s))")
fig5.show()
results5 = px.get_trendline_results(fig5)
results5.px_fit_results.iloc[0].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.997
Model:                            OLS   Adj. R-squared:                  0.997
Method:                 Least Squares   F-statistic:                 1.601e+05
Date:                Thu, 09 Dec 2021   Prob (F-statistic):               0.00
Time:                        12:08:40   Log-Likelihood:                 925.81
No. Observations:                 500   AIC:                            -1848.
Df Residuals:                     498   BIC:                            -1839.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        -15.1702      0.019   -793.889      0.000     -15.208     -15.133
x1             1.0313      0.003    400.085      0.000       1.026       1.036
==============================================================================
Omnibus:                       30.107   Durbin-Watson:                   1.925
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              104.060
Skew:                           0.034   Prob(JB):                     2.53e-23
Kurtosis:                       5.234   Cond. No.                         84.7
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [24]:
#Plotting
fig6 = px.scatter(x=np.log(cosine_df[cosine_df["numpy_naive_cosine_time"]>0]["n"]), y=np.log(cosine_df[cosine_df["numpy_naive_cosine_time"]>0]["numpy_naive_cosine_time"]), trendline="ols")
fig6.update_layout(title="Log-log Plot for Numpy Cosine",xaxis_title="log(vector length)", yaxis_title="log(time(s))")
fig6.show()
results6 = px.get_trendline_results(fig6)
results6.px_fit_results.iloc[0].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.480
Model:                            OLS   Adj. R-squared:                  0.479
Method:                 Least Squares   F-statistic:                     459.6
Date:                Thu, 09 Dec 2021   Prob (F-statistic):           1.01e-72
Time:                        12:08:40   Log-Likelihood:                 103.72
No. Observations:                 500   AIC:                            -203.4
Df Residuals:                     498   BIC:                            -195.0
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        -12.9962      0.099   -131.378      0.000     -13.191     -12.802
x1             0.2861      0.013     21.439      0.000       0.260       0.312
==============================================================================
Omnibus:                      325.466   Durbin-Watson:                   1.699
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2441.316
Skew:                           2.925   Prob(JB):                         0.00
Kurtosis:                      12.108   Cond. No.                         84.7
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

My results showed that the regression line for the log-log plot for Naive Cosine had the equation $y=1.0313x-15.1702$ whereas for Numpy it was $y=0.2861x-12.9962$. The Naive method has gradient slightly greater than 1, however there appear to be a couple of outliers above the line and it is barely greater than 1, and as such, I would conclude that this supports the theoretical conclusion that it has runtime of order n, with n being the vector length. For Numpy this is clearly the case that it is order n or less with a gradient of 0.2861. The constant for my computer for Naive Cosine here was $e^{-15.1702}=2.58027414\cdot10^{-7}$ and for Numpy it was $e^{-12.9962}=0.00000226893$.

#Theoretical and empirical analysis of the runtime of the cosine similarity measure applied directly to documents stored as sparse (dictionary) representations, and comparison with the dense representation version and Jaccard's algorithm

The following is my code for a sparse cosine similarity method taking arguments as dictionaries.

As it uses 2 non-nested for loops, for dictionaries size $n_1$ and $n_2$ with order $n$ where $n=max(n_1,n_2)$. This is comparable to both the Jaccard and dense cosine methods in being O(n). For two documents stored as dictionaries size $n_1$ and $n_2$, the $n$ for Jaccard would also be $n=max(n_1,n_2)$, and for dense cosine $n$ is the size of the vector representations for the two documents which, depending on the corpus used, is at the least $\frac{n_1+n_2}{2}$, and this will affect runtime for any given pair of documents.

In [25]:
#Sparse (dictionary) representation cosine method
def sparseCosine(C1,C2):
    num=0
    d1=0
    d2=0
    #Search for all key, value pairs in C1 dictionary
    for key,val in C1.items():
        #For C1 dot C1
        d1+=val*val
        #For C1 dot C2
        num+=val*C2.get(key,0)
    for val in C2.values():
        #For C2 dot C2
        d2+=val*val
    return num/(d1*d2)**0.5

This next code cell checks my sparse cosine method against the dense method for various values. If one didn't match, it would print a message stating this. It didn't, therefore I can conclude that the sparse method is correct.

In [26]:
#Test correctness of sparse against dense cosine
for i in range(0,len(doc_dicts),10):
  m_2=transpose(make_matrix([doc_dicts[i],doc_dicts[i+1]]))
  if sparseCosine(doc_dicts[i],doc_dicts[i+1]) != naiveCosine(m_2[0],m_2[1]):
    print(f"The answers for Sparse Cosine and Dense Cosine for docs {i} and {i+1} don't match")

In [27]:
#Time sparse cosine
sparse_ns=[]
sparse_cosine_times=[]
sparse_cosine_errors=[]

for i in range(500):
  c1=random.choice(doc_dicts)
  c2=random.choice(doc_dicts)
  sparse_ns.append(max(len(c1),len(c2)))
  (ans_cos,mean_cos,error_cos)=timeit(sparseCosine,c1,c2,repeats=100)
  sparse_cosine_times.append(mean_cos)
  sparse_cosine_errors.append(error_cos)
    
sparse_cosine_dict={'n':sparse_ns, 'sparse_cosine_time':sparse_cosine_times, 'sparse_cosine_error':sparse_cosine_errors}    
sparse_cosine_df=pd.DataFrame(sparse_cosine_dict)

Here I plot the mean time taken for 100 repeats of my sparse cosine algorithm on randomly selected dictionaries of various sizes, followed by an overlay of the two dense versions. For Sparse I take n to be the larger dictionary size, whereas for the dense algorithms I take it to be vector size. I have not overlayed it against my implementation of Jaccard as the plots would overlay too closely, and not much difference would be seen, as you can see from the individual plots, however I will provide analysis comparing the order and constants following my log-log plots.

In [28]:
#Plotting
fig7 = px.scatter(sparse_cosine_df, x="n", y="sparse_cosine_time")
fig7.update_layout(title="Sparse Cosine Algorithm",xaxis_title="dictionary length", yaxis_title="time(s)")
fig7.show()
fig4.add_trace(go.Scatter(x=sparse_cosine_df["n"], y=sparse_cosine_df["sparse_cosine_time"],mode='markers',name='sparse cosine'))
fig4.update_layout(title="Comparison of Naive, Numpy and Sparse Cosine Algorithms",xaxis_title="vector(/dictionary for sparse) length", yaxis_title="time(s)")
fig4.show()

Sparse Cosine appears to be faster than dense cosine (not the Numpy version). Below is the log-log plot for the Sparse version and the OLS regression results.

In [29]:
#Plotting
fig8 = px.scatter(x=np.log(sparse_cosine_df["n"]), y=np.log(sparse_cosine_df["sparse_cosine_time"]), trendline="ols")
fig8.update_layout(title="Log-log Plot for Sparse Cosine Algorithm",xaxis_title="log(n)", yaxis_title="log(time(s))")
fig8.show()
results8 = px.get_trendline_results(fig8)
results8.px_fit_results.iloc[0].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.546
Model:                            OLS   Adj. R-squared:                  0.545
Method:                 Least Squares   F-statistic:                     597.9
Date:                Thu, 09 Dec 2021   Prob (F-statistic):           2.45e-87
Time:                        12:09:18   Log-Likelihood:                -143.21
No. Observations:                 500   AIC:                             290.4
Df Residuals:                     498   BIC:                             298.8
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        -16.6198      0.380    -43.768      0.000     -17.366     -15.874
x1             1.2026      0.049     24.452      0.000       1.106       1.299
==============================================================================
Omnibus:                      206.203   Durbin-Watson:                   2.089
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              659.057
Skew:                          -1.998   Prob(JB):                    7.72e-144
Kurtosis:                       6.958   Cond. No.                         206.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

My results showed that the regression line for the log-log plot for Sparse Cosine had the equation $y=1.2026x-16.6198	$. Similarly to the Jaccard method, it therefore has gradient significantly greater than 1 and as such, doesn't support the theoretical conclusion that it has a runtime of order n or less, with n being the size of the larger dictionary compared. The constant for my computer for Sparse Cosine here was $e^{-15.9951}=6.054978\cdot10^{-8}$. My dense cosine and Jaccard algorithm had log-log plot OLS regression results with best fit lines $y=1.0313x-15.1702$ and $y=1.2141x−16.2890$ respectively. I would conclude that each outperform the Sparse method as values for the dictionary or vector size get large enough, due to the lower gradients, however at smaller dictionary sizes, the constants for my machine ($8.42902194\cdot10^{-8}$ for Jaccard and $2.58027414\cdot10^{-7}$ for dense cosine) suggest that Sparse Cosine might outperform dense but maybe not Jaccard. Having said that, it must be noted that dense cosine has the added requirement of collecting all words from all documents to be compared. As such without creating a corpus, or list of all possible keys, in advance, this algorithm may take more time to actually implement if on collections of documents if we start from a list of dictionaries representation and build our corpus from there.

#Theoretical analysis of the runtime of an all-pairs similarity comparison algorithm, taking documents stored as dictionaries and a chosen similarity measure to be applied as arguments

Theoretically, an all pairs similarities algorithm would be of order $O(m^2n)$ where $m$ is the number of documents used, and $n$ is dependent on the similarity measure used, as described for the similarity comparisons above. For other similarity measures it would be $O(m^2f(n))$ where f is the time bound function of the similarity measure with $n$ being at most, the maximum length of the dictionaries or vectors being compared.

The $m^2$ term is because (Weisstein, E.W.) choosing $2$ dictionaries from a list of length $m$ is as follows:

$C_2^m=\frac{m!}{2!(m-2)!}=\frac{m(m-1)(m-2)(m-3)...\cdot2\cdot1}{2!(m-2)(m-3)...\cdot2\cdot1}=\frac{m(m-1)(m-2)(m-3)...\cdot2\cdot1}{2!(m-2)(m-3)...\cdot2\cdot1}=\frac{m(m-1)}{2}=\frac{m^2-m}{2}<m^2$

The following code is for a method accepting a set of documents as a list of dictionaries and a chosen similarity measure as an argument. I have actually written this such that it can accept other variable types such as numpy arrays instead of a list of dictionaries, and as such this method can run the numpy dense cosine algorithm I wrote earlier. I have provided times taken to run Jaccard, Sparse Cosine and Numpy Dense Cosine for all pairs of 11 documents. This supports the theoretical notion that the similarity measure does affect the running time for computing all pairs similarities, and as such the constant for the all pairs algorithm for the machine the all pairs similarity comparison is being run on. The mean time taken for 11 documents for the Jaccard method was $0.08889164447784424$ seconds, which implies that for 200k documents, it would take $\frac{200000^2 \cdot  0.08889164447784424}{11^2}=29385667.595981568$ seconds or approximately 11.18 months, whereas for Sparse Cosine it was $0.057260022163391114$ seconds, which implies that for 200k documents, it would take $\frac{200000^2 \cdot  0.057260022163391114}{11^2}=18928932.946575575$ seconds or approximately 7.20 months. The mean time taken for 11 documents for the Numpy Cosine method was $0.0018838953971862793$ seconds, which implies that for 200k documents, it would take $\frac{200000^2 \cdot  0.0018838953971862793}{11^2}=622775.3379128196$ seconds or approximately a week. These times would increase if we used longer dictionary sizes for the list of dictionary methods and if we used a larger corpus for the vector representation method.

In [30]:
#All-pairs similarity comparison algorithm
def compareAllDocSimilarities(list_of_dicts, sim_measure):
    sims_dict_out={}
    
    #Loop through once to get first document
    for i in range(len(list_of_dicts)-1):
        #Loop through all documents after first document in list
        #Ensures we don't compare documents to themselves or to another document twice
        for j in range(i+1, len(list_of_dicts)):
            sims_dict_out[(i,j)]=sim_measure(list_of_dicts[i],list_of_dicts[j])
            #Store similarities in a dictionary with key as (i,j) tuple
    
    #Return dictionary as output
    #To look up similarity of docs in position a,b in doc list with a<b from result dictionary, use key (a,b)
    #If a>=b, no result will be found
    return sims_dict_out

In [77]:
#Time for each method on collection of 11 documents
(ans_jaccard,mean_jaccard,error_jaccard)=timeit(compareAllDocSimilarities,doc_dicts[979:989],jaccard,repeats=100)
(ans_sparse_cosine,mean_sparse_cosine,error_sparse_cosine)=timeit(compareAllDocSimilarities,doc_dicts[979:989],sparseCosine,repeats=100)
print(f"The mean time taken to compare all pairs of the last 11 documents from my data with Jaccard was {mean_jaccard}")
print(f"The mean time taken to compare all pairs of the last 11 documents from my data with Sparse Cosine was {mean_sparse_cosine}")
(ans_dense_cosine,mean_dense_cosine,error_dense_cosine)=timeit(compareAllDocSimilarities,np.array(transpose(make_matrix(doc_dicts[979:989]))),naiveCosineNumpyDot,repeats=100)
print(f"The mean time taken to compare all pairs of the last 11 documents from my data with Numpy Dense Cosine was {mean_dense_cosine}")
if ans_sparse_cosine != ans_dense_cosine:
  print("The results for comparing all pairs of documents with dense and sparse cosine do not match, therefore the all pairs algorithm has an error")

The mean time taken to compare all pairs of the last 11 documents from my data with Jaccard was 0.08889164447784424
The mean time taken to compare all pairs of the last 11 documents from my data with Sparse Cosine was 0.057260022163391114
The mean time taken to compare all pairs of the last 11 documents from my data with Numpy Dense Cosine was 0.0018838953971862793


In [85]:
200000**2*0.0018838953971862793/11**2

622775.3379128196

#Empirical analysis of the runtime of a parallel processing version of the all-pairs similarity comparison algorithm detailed above

The following code implements a parallel processing version of the all pairs similarities algorithm, using Map from MapReduce to assign different similarity calculations to different processes, but no reduce algorithm. This is followed by a successful test for correctness against the original all pairs algorithm and a plot of time taken to compare all documents vs document collection size, and two log-log plots.

In [54]:
from collections import defaultdict
from multiprocessing import Pool
import sim_map_reduce as smr

#Map parallel method, adapted from Dr Adam Barrett's lab MapReduce algorithm
def map_parallel(inputs,mapper,mapprocesses=3):
    collector={}
    mappool = Pool(processes=mapprocesses)
    
    mapresults=mappool.map(mapper,inputs)
    mappool.close()
    
    for mapresult in mapresults:
        for (key, value) in mapresult:     #pass each input to the mapper function and receive back each key,value pair yielded
            collector[key]=value
    
    return collector

#My parallel all-pairs comparison method
def all_pairs_parallel(list_of_dicts, sim_measure, mapprocesses=3):
  inputs=[]
  #Create inputs for map_parallel
  for i in range(len(list_of_dicts)-1):
        for j in range(i+1, len(list_of_dicts)):
          inputs.append((i,j,list_of_dicts[i],list_of_dicts[j],sim_measure))
  
  #Calculate similarities
  outputs=map_parallel(inputs,smr.sim_mapper,mapprocesses)
  return outputs

In [78]:
#Correctness test agains non-parallel all-pairs comparison
ans_all_pairs=compareAllDocSimilarities(doc_dicts[0:10],jaccard)
ans_map_sims=all_pairs_parallel(doc_dicts[0:10],jaccard, mapprocesses=1)
for key in ans_all_pairs.keys():
  if ans_all_pairs.get(key) != ans_map_sims.get(key):
    print(f"Jaccard similarity measures of the all pairs and the map all pairs algorithms of bags {key} don't match")

In [79]:
#Time parallel all-pairs algorithm
map_ns=[]
map_times=[]
map_errors=[]

for i in range(10,100,10):
  map_ns.append(i)
  (ans_map,mean_map,error_map)=timeit(all_pairs_parallel,doc_dicts[0:i],jaccard, mapprocesses=1,repeats=5)
  map_times.append(mean_map)
  map_errors.append(error_map)
    
map_dict={'n':map_ns, 'map_time':map_times, 'map_error':map_errors}    
map_df=pd.DataFrame(map_dict)

In [80]:
#Plotting
fig9 = px.scatter(map_df, x="n", y="map_time")
fig9.update_layout(title="Map All Pairs Comparison Algorithm",xaxis_title="number of docs to compare", yaxis_title="time(s)")
fig9.show()

In [81]:
#Plotting
fig10 = px.scatter(x=np.log(map_df[map_df["n"]>0]["n"]), y=np.log(map_df[map_df["n"]>0]["map_time"]), trendline="ols")
fig10.update_layout(title="Map All Pairs Log-log plot",xaxis_title="log(number of docs to compare)", yaxis_title="log(time(s))")
fig10.show()

In [82]:
#Plotting
fig11 = px.scatter(x=np.log(map_df[map_df["n"]>20]["n"]), y=np.log(map_df[map_df["n"]>20]["map_time"]), trendline="ols")
fig11.update_layout(title="Map All Pairs Log-log plot (number of docs > 20)",xaxis_title="log(number of docs to compare)", yaxis_title="log(time(s))")
fig11.show()
results11 = px.get_trendline_results(fig11)
results11.px_fit_results.iloc[0].summary()

/usr/local/lib/python3.7/dist-packages/statsmodels/stats/stattools.py:71: ValueWarning:

omni_normtest is not valid with less than 8 observations; 7 samples were given.



<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.950
Model:                            OLS   Adj. R-squared:                  0.940
Method:                 Least Squares   F-statistic:                     95.23
Date:                Thu, 09 Dec 2021   Prob (F-statistic):           0.000192
Time:                        18:04:54   Log-Likelihood:                 3.3195
No. Observations:                   7   AIC:                            -2.639
Df Residuals:                       5   BIC:                            -2.747
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -8.6282      0.751    -11.485      0.000     -10.559      -6.697
x1             1.8107      0.186      9.758      0.000       1.334       2.288
==============================================================================
Omnibus:                          nan   Durbin-Watson:                   1.061
Prob(Omnibus):                    nan   Jarque-Bera (JB):                1.121
Skew:                           0.899   Prob(JB):                        0.571
Kurtosis:                       2.219   Cond. No.                         47.9
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In the first log-log plot above for the Map all pairs algorithm, smaller collections of documents seemed to skew the results. As such, I removed smaller collection sizes to provide a more accurate empirical estimate of the complexity for large m, with m being the number of documents to compare. In doing so, I found a gradient of 1.8107 as opposed to 1.0639, supporting the notion that it is order $m^2$, assuming the time taken to complete a given similarity measure on a collection of documents with a known maximum length. Otherwise it would be $O(m^2n)$ with n being the maximum dictionary or vector length for the respective algorithm. The constant for my machine for comparing all pairs for Jaccard method was $e^{-8.6282}=0.00017898653$.

The following times(s) are mean times taken to compare 11 documents with the map all pairs algorithm using Jaccard's method with 1-4 map processes respectively. The algorithm gets slower each time, implying that 1 process is optimal for collections of documents of this size. Perhaps we would see an improvement for larger collections, as the time taken to set up the multiprocessing may take longer than the time saved for collections of this size, however I am also using colab, which only allows access to 1 CPU. As such, it won't be faster than the previous all-pairs method unless run on a machine with multiple CPUs.

In [87]:
#Test parallel all-pairs for Jaccard Min First with different number of processes
(ans_mr_jaccard,mean_mr_jaccard,error_mr_jaccard)=timeit(all_pairs_parallel,doc_dicts[0:11],jaccard,mapprocesses=1,repeats=100)
print(mean_mr_jaccard)
(ans_mr_jaccard,mean_mr_jaccard,error_mr_jaccard)=timeit(all_pairs_parallel,doc_dicts[0:11],jaccard,mapprocesses=2,repeats=100)
print(mean_mr_jaccard)
(ans_mr_jaccard,mean_mr_jaccard,error_mr_jaccard)=timeit(all_pairs_parallel,doc_dicts[0:11],jaccard,mapprocesses=3,repeats=100)
print(mean_mr_jaccard)
(ans_mr_jaccard,mean_mr_jaccard,error_mr_jaccard)=timeit(all_pairs_parallel,doc_dicts[0:11],jaccard,mapprocesses=4,repeats=100)
print(mean_mr_jaccard)

0.0749589729309082
0.09386156558990479
0.12494329929351806
0.16119611024856567


#Bibliography
Yanchii, Y. (2019). *Beating NumPy performance by extending Python with C*. [online] Analytics Vidhya. Available at: https://medium.com/analytics-vidhya/beating-numpy-performance-by-extending-python-with-c-c9b644ee2ca8 [Accessed 9 Dec. 2021].

Weisstein, E.W. (n.d.). Binomial Coefficient. [online] mathworld.wolfram.com. Available at: https://mathworld.wolfram.com/BinomialCoefficient.html.

Tas, S. (2021). Faster Lookups In Python. [online] Medium. Available at: https://towardsdatascience.com/faster-lookups-in-python-1d7503e9cd38#:~:text=Lookups%20are%20faster%20in%20dictionaries [Accessed 9 Dec. 2021].